# Project testing

# Table of Contents
* [Project testing](#Project-testing)
	* [Packages](#Packages)
	* [Testing idoms](#Testing-idoms)
		* [single functions](#single-functions)
		* [generators](#generators)
		* [classes](#classes)
		* [module fixtures](#module-fixtures)
		* [decorator fixtures](#decorator-fixtures)
	* [How tests fail](#How-tests-fail)
		* [by assertion](#by-assertion)
		* [by exceptions](#by-exceptions)
			* [no tests found??](#no-tests-found??)
			* [Something crazy with the decorator.](#Something-crazy-with-the-decorator.)
		* [by crashing nose](#by-crashing-nose)


## Packages

```
(base)> conda create -n ipy_tests python=3 numpy jupyter nose pip
(base)> conda activate ipy_tests
(ipy_tests)> pip install git+https://github.com/datacamp/ipython_nose@v0.4.1
(ipy_tests)> jupyter notebook testing.ipynb
```

In [1]:
%load_ext ipython_nose
import sys
!{sys.exec_prefix}/bin/pip show ipython_nose

Name: ipython-nose
Version: 0.4.1
Summary: IPython extension to run nosetests against the current kernel.
Home-page: https://github.com/taavi/ipython_nose
Author: Taavi Burns <taavi at taaviburns dot ca>, Greg Ward <greg at gerg dot ca>
Author-email: UNKNOWN
License: README.rst
Location: /Users/adefusco/Applications/miniconda3/envs/ipy_tests/lib/python3.6/site-packages
Requires: 
You are using pip version 9.0.1, however version 9.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import numpy as np
np.random.seed(42)

## Testing idoms

http://pythontesting.net/framework/nose/nose-fixture-reference/

Instructions
-----------
Create a NumPy array of 5 integers where the values are all greater than 5.

In [3]:
ints = np.random.randint(6, 10, size=5)
ints

array([8, 9, 6, 8, 8])

### single functions

These are examples of single function tests.

In [4]:
%%nose

def test_dtype():
    d = ints.dtype
    assert d == np.int, 'Your array has the wrong dtype.'

def test_values():
    assert (ints > 5).all(), 'Every value should be greater than 5'

2/2 tests passed


### generators

Alternatively, a generator function can be used to execute multiple tests.

In [5]:
%%nose

def test_dtype():
    d = ints.dtype
    assert d == np.int, 'Your array has the wrong dtype.'

def test_values():
    def _test_value(v, threshold=5):
        assert v > threshold, '{} is not greater than {}'.format(v, threshold)
        
    for i in ints:
        yield _test_value, i

6/6 tests passed


### classes

A class may be used to initialize objects and reuse them in multiple tests.

Per-test initialization
----------------------

`setup()` **must be a classmethod**. It will be run 6 times, once before each test.

In [6]:
%%nose

class TestValuesSetup():
    @classmethod
    def setup(self):
        self.threshold = 5
        self.dtype = ints.dtype
    
    def test_dtype(self):
        assert self.dtype == np.int, 'Your array has the wrong dtype.'
    
    def test_values(self):
        def _test_value(v):
            assert v > self.threshold, '{} is not greater than {}'.format(v, self.threshold)
            
        for i in ints:
            yield _test_value, i

6/6 tests passed


Single initialization
--------------------

The `setup_class()` function is only run once and the state used for all 6 tests. There are caveats to using this idiom.

In [7]:
%%nose

class TestValuesSetupClass():
    @classmethod
    def setup_class(self):
        self.threshold = 5
        self.dtype = ints.dtype
    
    def test_dtype(self):
        assert self.dtype == np.int, 'Your array has the wrong dtype.'
    
    def test_values(self):
        def _test_value(v):
            assert v > self.threshold, '{} is not greater than {}'.format(v, self.threshold)
            
        for i in ints:
            yield _test_value, i

6/6 tests passed


### module fixtures

This looks ugly and probably not recommended. We'll see that this behaves like `setup_class()` above and has the same caveats.

In [8]:
%%nose

def setup_module():
    global d, threshold
    d = ints.dtype
    threshold = 5
    
def test_dtype():
    assert d == np.int, 'Your array has the wrong dtype.'

def test_values():
    def _test_value(v):
        assert v > threshold, '{} is not greater than {}'.format(v, threshold)
        
    for i in ints:
        yield _test_value, i

6/6 tests passed


### decorator fixtures

Just like module fixtures this looks ugly

In [9]:
%%nose

from nose.tools import with_setup

def setup_tests():
    global d, threshold
    d = ints.dtype
    threshold = 5
    
@with_setup(setup_tests)
def test_dtype():
    assert d == np.int, 'Your array has the wrong dtype.'

@with_setup(setup_tests)
def test_values():
    def _test_value(v):
        assert v > threshold, '{} is not greater than {}'.format(v, threshold)
        
    for i in ints:
        yield _test_value, i

7/7 tests passed


## How tests fail

### by assertion

The test-taker did not follow the instructions and some values are less than 5.

In [10]:
ints = np.random.randint(0, 10, size=5)
ints

array([7, 4, 6, 9, 2])

The three testing idioms fail on the `assert` lines. All tests are recorded as having been run for each idiom.

In [11]:
%%nose

def test_dtype():
    d = ints.dtype
    assert d == np.int, 'Your array has the wrong dtype.'

def test_values():
    assert (ints > 5).all(), 'Every value should be greater than 5'

1/2 tests passed; 1 failed
__main__.test_values
Traceback (most recent call last):
  File "/Users/adefusco/Applications/miniconda3/envs/ipy_tests/lib/python3.6/unittest/case.py", line 59, in testPartExecutor
    yield
  File "/Users/adefusco/Applications/miniconda3/envs/ipy_tests/lib/python3.6/unittest/case.py", line 605, in run
    testMethod()
  File "/Users/adefusco/Applications/miniconda3/envs/ipy_tests/lib/python3.6/site-packages/nose/case.py", line 197, in runTest
    self.test(*self.arg)
  File "<string>", line 7, in test_values
AssertionError: Every value should be greater than 5



In [12]:
%%nose

def test_dtype():
    d = ints.dtype
    assert d == np.int, 'Your array has the wrong dtype.'

def test_values():
    def _test_value(v, threshold=5):
        assert v > threshold, '{} is not greater than {}'.format(v, threshold)
        
    for i in ints:
        yield _test_value, i

4/6 tests passed; 2 failed
__main__.test_values(4,)
Traceback (most recent call last):
  File "/Users/adefusco/Applications/miniconda3/envs/ipy_tests/lib/python3.6/unittest/case.py", line 59, in testPartExecutor
    yield
  File "/Users/adefusco/Applications/miniconda3/envs/ipy_tests/lib/python3.6/unittest/case.py", line 605, in run
    testMethod()
  File "/Users/adefusco/Applications/miniconda3/envs/ipy_tests/lib/python3.6/site-packages/nose/case.py", line 197, in runTest
    self.test(*self.arg)
  File "<string>", line 8, in _test_value
AssertionError: 4 is not greater than 5

__main__.test_values(2,)
Traceback (most recent call last):
  File "/Users/adefusco/Applications/miniconda3/envs/ipy_tests/lib/python3.6/unittest/case.py", line 59, in testPartExecutor
    yield
  File "/Users/adefusco/Applications/miniconda3/envs/ipy_tests/lib/python3.6/unittest/case.py", line 605, in run
    testMethod()
  File "/Users/adefusco/Applications/miniconda3/envs/ipy_tests/lib/python3.6/site-packag

In [13]:
%%nose

class TestValuesSetup():
    @classmethod
    def setup(self):
        self.threshold = 5
        self.dtype = ints.dtype
    
    def test_dtype(self):
        assert self.dtype == np.int, 'Your array has the wrong dtype.'
    
    def test_values(self):
        def _test_value(v):
            assert v > self.threshold, '{} is not greater than {}'.format(v, self.threshold)
            
        for i in ints:
            yield _test_value, i

4/6 tests passed; 2 failed
__main__.TestValuesSetup.test_values(4,)
Traceback (most recent call last):
  File "/Users/adefusco/Applications/miniconda3/envs/ipy_tests/lib/python3.6/unittest/case.py", line 59, in testPartExecutor
    yield
  File "/Users/adefusco/Applications/miniconda3/envs/ipy_tests/lib/python3.6/unittest/case.py", line 605, in run
    testMethod()
  File "/Users/adefusco/Applications/miniconda3/envs/ipy_tests/lib/python3.6/site-packages/nose/case.py", line 197, in runTest
    self.test(*self.arg)
  File "<string>", line 13, in _test_value
AssertionError: 4 is not greater than 5

__main__.TestValuesSetup.test_values(2,)
Traceback (most recent call last):
  File "/Users/adefusco/Applications/miniconda3/envs/ipy_tests/lib/python3.6/unittest/case.py", line 59, in testPartExecutor
    yield
  File "/Users/adefusco/Applications/miniconda3/envs/ipy_tests/lib/python3.6/unittest/case.py", line 605, in run
    testMethod()
  File "/Users/adefusco/Applications/miniconda3/envs/ip

In [14]:
%%nose

class TestValuesSetupClass():
    @classmethod
    def setup_class(self):
        self.threshold = 5
        self.dtype = ints.dtype
    
    def test_dtype(self):
        assert self.dtype == np.int, 'Your array has the wrong dtype.'
    
    def test_values(self):
        def _test_value(v):
            assert v > self.threshold, '{} is not greater than {}'.format(v, self.threshold)
            
        for i in ints:
            yield _test_value, i

4/6 tests passed; 2 failed
__main__.TestValuesSetupClass.test_values(4,)
Traceback (most recent call last):
  File "/Users/adefusco/Applications/miniconda3/envs/ipy_tests/lib/python3.6/unittest/case.py", line 59, in testPartExecutor
    yield
  File "/Users/adefusco/Applications/miniconda3/envs/ipy_tests/lib/python3.6/unittest/case.py", line 605, in run
    testMethod()
  File "/Users/adefusco/Applications/miniconda3/envs/ipy_tests/lib/python3.6/site-packages/nose/case.py", line 197, in runTest
    self.test(*self.arg)
  File "<string>", line 13, in _test_value
AssertionError: 4 is not greater than 5

__main__.TestValuesSetupClass.test_values(2,)
Traceback (most recent call last):
  File "/Users/adefusco/Applications/miniconda3/envs/ipy_tests/lib/python3.6/unittest/case.py", line 59, in testPartExecutor
    yield
  File "/Users/adefusco/Applications/miniconda3/envs/ipy_tests/lib/python3.6/unittest/case.py", line 605, in run
    testMethod()
  File "/Users/adefusco/Applications/minicond

In [15]:
%%nose

def setup_module():
    global d, threshold
    d = ints.dtype
    threshold = 5
    
def test_dtype():
    assert d == np.int, 'Your array has the wrong dtype.'

def test_values():
    def _test_value(v):
        assert v > threshold, '{} is not greater than {}'.format(v, threshold)
        
    for i in ints:
        yield _test_value, i

4/6 tests passed; 2 failed
__main__.test_values(4,)
Traceback (most recent call last):
  File "/Users/adefusco/Applications/miniconda3/envs/ipy_tests/lib/python3.6/unittest/case.py", line 59, in testPartExecutor
    yield
  File "/Users/adefusco/Applications/miniconda3/envs/ipy_tests/lib/python3.6/unittest/case.py", line 605, in run
    testMethod()
  File "/Users/adefusco/Applications/miniconda3/envs/ipy_tests/lib/python3.6/site-packages/nose/case.py", line 197, in runTest
    self.test(*self.arg)
  File "<string>", line 12, in _test_value
AssertionError: 4 is not greater than 5

__main__.test_values(2,)
Traceback (most recent call last):
  File "/Users/adefusco/Applications/miniconda3/envs/ipy_tests/lib/python3.6/unittest/case.py", line 59, in testPartExecutor
    yield
  File "/Users/adefusco/Applications/miniconda3/envs/ipy_tests/lib/python3.6/unittest/case.py", line 605, in run
    testMethod()
  File "/Users/adefusco/Applications/miniconda3/envs/ipy_tests/lib/python3.6/site-packa

In [16]:
%%nose

from nose.tools import with_setup

def setup_tests():
    global d, threshold
    d = ints.dtype
    threshold = 5
    
@with_setup(setup_tests)
def test_dtype():
    assert d == np.int, 'Your array has the wrong dtype.'

@with_setup(setup_tests)
def test_values():
    def _test_value(v):
        assert v > threshold, '{} is not greater than {}'.format(v, threshold)
        
    for i in ints:
        yield _test_value, i

5/7 tests passed; 2 failed
__main__.test_values(4,)
Traceback (most recent call last):
  File "/Users/adefusco/Applications/miniconda3/envs/ipy_tests/lib/python3.6/unittest/case.py", line 59, in testPartExecutor
    yield
  File "/Users/adefusco/Applications/miniconda3/envs/ipy_tests/lib/python3.6/unittest/case.py", line 605, in run
    testMethod()
  File "/Users/adefusco/Applications/miniconda3/envs/ipy_tests/lib/python3.6/site-packages/nose/case.py", line 197, in runTest
    self.test(*self.arg)
  File "<string>", line 16, in _test_value
AssertionError: 4 is not greater than 5

__main__.test_values(2,)
Traceback (most recent call last):
  File "/Users/adefusco/Applications/miniconda3/envs/ipy_tests/lib/python3.6/unittest/case.py", line 59, in testPartExecutor
    yield
  File "/Users/adefusco/Applications/miniconda3/envs/ipy_tests/lib/python3.6/unittest/case.py", line 605, in run
    testMethod()
  File "/Users/adefusco/Applications/miniconda3/envs/ipy_tests/lib/python3.6/site-packa

### by exceptions

http://pythontesting.net/framework/unittest/when-unittest-fixtures-fail/

The test-taker broke both rules, values are less than 5 and `ints` is a list.

In [17]:
ints = np.random.randint(0, 10, size=5).tolist()

The two test functions fail appropriately because
* `.dtype` is not an attribute of lists.
* `>` comparison is not supported.

In [18]:
%%nose

def test_dtype():
    d = ints.dtype
    assert d == np.int, 'Your arrays has the wrong dtype.'

def test_values():
    assert (ints > 5).all(), 'Every value should be greater than 5'

0/2 tests passed; 2 failed
__main__.test_dtype
Traceback (most recent call last):
  File "/Users/adefusco/Applications/miniconda3/envs/ipy_tests/lib/python3.6/unittest/case.py", line 59, in testPartExecutor
    yield
  File "/Users/adefusco/Applications/miniconda3/envs/ipy_tests/lib/python3.6/unittest/case.py", line 605, in run
    testMethod()
  File "/Users/adefusco/Applications/miniconda3/envs/ipy_tests/lib/python3.6/site-packages/nose/case.py", line 197, in runTest
    self.test(*self.arg)
  File "<string>", line 3, in test_dtype
AttributeError: 'list' object has no attribute 'dtype'

__main__.test_values
Traceback (most recent call last):
  File "/Users/adefusco/Applications/miniconda3/envs/ipy_tests/lib/python3.6/unittest/case.py", line 59, in testPartExecutor
    yield
  File "/Users/adefusco/Applications/miniconda3/envs/ipy_tests/lib/python3.6/unittest/case.py", line 605, in run
    testMethod()
  File "/Users/adefusco/Applications/miniconda3/envs/ipy_tests/lib/python3.6/site-p

Some tests pass because the generator was written in a container-agnostic fashion.

In [19]:
%%nose

def test_dtype():
    d = ints.dtype
    assert d == np.int, 'Your array has the wrong dtype.'

def test_values():
    def _test_value(v, threshold=5):
        assert v > threshold, '{} is not greater than {}'.format(v, threshold)
        
    for i in ints:
        yield _test_value, i

3/6 tests passed; 3 failed
__main__.test_dtype
Traceback (most recent call last):
  File "/Users/adefusco/Applications/miniconda3/envs/ipy_tests/lib/python3.6/unittest/case.py", line 59, in testPartExecutor
    yield
  File "/Users/adefusco/Applications/miniconda3/envs/ipy_tests/lib/python3.6/unittest/case.py", line 605, in run
    testMethod()
  File "/Users/adefusco/Applications/miniconda3/envs/ipy_tests/lib/python3.6/site-packages/nose/case.py", line 197, in runTest
    self.test(*self.arg)
  File "<string>", line 3, in test_dtype
AttributeError: 'list' object has no attribute 'dtype'

__main__.test_values(4,)
Traceback (most recent call last):
  File "/Users/adefusco/Applications/miniconda3/envs/ipy_tests/lib/python3.6/unittest/case.py", line 59, in testPartExecutor
    yield
  File "/Users/adefusco/Applications/miniconda3/envs/ipy_tests/lib/python3.6/unittest/case.py", line 605, in run
    testMethod()
  File "/Users/adefusco/Applications/miniconda3/envs/ipy_tests/lib/python3.6/si

All tests are run and nothing passes because `setup()` is run each time.

In [20]:
%%nose

class TestValuesSetup():
    @classmethod
    def setup(self):
        self.threshold = 5
        self.dtype = ints.dtype
    
    def test_dtype(self):
        assert self.dtype == np.int, 'Your array has the wrong dtype.'
    
    def test_values(self):
        def _test_value(v):
            assert v > self.threshold, '{} is not greater than {}'.format(v, self.threshold)
            
        for i in ints:
            yield _test_value, i

0/6 tests passed; 6 failed
__main__.TestValuesSetup.test_dtype
Traceback (most recent call last):
  File "/Users/adefusco/Applications/miniconda3/envs/ipy_tests/lib/python3.6/unittest/case.py", line 59, in testPartExecutor
    yield
  File "/Users/adefusco/Applications/miniconda3/envs/ipy_tests/lib/python3.6/unittest/case.py", line 601, in run
    self.setUp()
  File "/Users/adefusco/Applications/miniconda3/envs/ipy_tests/lib/python3.6/site-packages/nose/case.py", line 381, in setUp
    try_run(self.inst, ('setup', 'setUp'))
  File "/Users/adefusco/Applications/miniconda3/envs/ipy_tests/lib/python3.6/site-packages/nose/util.py", line 471, in try_run
    return func()
  File "<string>", line 6, in setup
AttributeError: 'list' object has no attribute 'dtype'

__main__.TestValuesSetup.test_values(6,)
Traceback (most recent call last):
  File "/Users/adefusco/Applications/miniconda3/envs/ipy_tests/lib/python3.6/unittest/case.py", line 59, in testPartExecutor
    yield
  File "/Users/adefus

#### no tests found??

Because `setup_class()` failed, the class itself was not built and **no tests were even discovered.**

I believe this to be expected behavior, but unfortunate.

In [21]:
%%nose

class TestValuesSetupClass():
    @classmethod
    def setup_class(self):
        self.threshold = 5
        self.dtype = ints.dtype
    
    def test_dtype(self):
        assert self.dtype == np.int, 'Your array has the wrong dtype.'
    
    def test_values(self):
        def _test_value(v):
            assert v > self.threshold, '{} is not greater than {}'.format(v, self.threshold)
            
        for i in ints:
            yield _test_value, i

No tests found.

Again, `setup_module()` failed and nothing happened.

In [22]:
%%nose

def setup_module():
    global d, threshold
    d = ints.dtype
    threshold = 5
    
def test_dtype():
    assert d == np.int, 'Your array has the wrong dtype.'

def test_values():
    def _test_value(v):
        assert v > threshold, '{} is not greater than {}'.format(v, threshold)
        
    for i in ints:
        yield _test_value, i

No tests found.

`ValuesTestSetup` does not conform to the nose regex.

In [23]:
%%nose

class ValuesTestSetup():
    @classmethod
    def setup(self):
        self.threshold = 5
        self.dtype = ints.dtype
    
    def test_dtype(self):
        assert self.dtype == np.int, 'Your array has the wrong dtype.'
    
    def test_values(self):
        def _test_value(v):
            assert v > self.threshold, '{} is not greater than {}'.format(v, self.threshold)
            
        for i in ints:
            yield _test_value, i

No tests found.

#### Something crazy with the decorator.

What does `-1/2` tests pass mean??

In [24]:
%%nose

from nose.tools import with_setup

def setup_tests():
    global d, threshold
    d = ints.dtype
    threshold = 5
    
@with_setup(setup_tests)
def test_dtype():
    assert d == np.int, 'Your array has the wrong dtype.'

@with_setup(setup_tests)
def test_values():
    def _test_value(v):
        assert v > threshold, '{} is not greater than {}'.format(v, threshold)
        
    for i in ints:
        yield _test_value, i

-1/2 tests passed; 3 failed
__main__.setup_tests
Traceback (most recent call last):
  File "/Users/adefusco/Applications/miniconda3/envs/ipy_tests/lib/python3.6/unittest/case.py", line 59, in testPartExecutor
    yield
  File "/Users/adefusco/Applications/miniconda3/envs/ipy_tests/lib/python3.6/unittest/case.py", line 605, in run
    testMethod()
  File "/Users/adefusco/Applications/miniconda3/envs/ipy_tests/lib/python3.6/site-packages/nose/case.py", line 197, in runTest
    self.test(*self.arg)
  File "<string>", line 6, in setup_tests
AttributeError: 'list' object has no attribute 'dtype'

__main__.test_dtype
Traceback (most recent call last):
  File "/Users/adefusco/Applications/miniconda3/envs/ipy_tests/lib/python3.6/unittest/case.py", line 59, in testPartExecutor
    yield
  File "/Users/adefusco/Applications/miniconda3/envs/ipy_tests/lib/python3.6/unittest/case.py", line 601, in run
    self.setUp()
  File "/Users/adefusco/Applications/miniconda3/envs/ipy_tests/lib/python3.6/site

### by crashing nose

Here we will break only one rule.

In [25]:
ints = np.random.randint(6, 10, size=5).tolist()

Poorly written test cells can cause a full crash of nose and ipython_nose.

Generally this means any uncaught exceptions outside of test functions or classes.

On DataCamp this AttributeError leads to a *Timeout* when running `Check Project`.

In [26]:
%%nose

d = ints.dtype

def test_dtype():
    assert d == np.int, 'Your arrays has the wrong dtype.'

def test_values():
    assert (ints > 5).all(), 'Every value should be greater than 5'

AttributeError: 'list' object has no attribute 'dtype'

The `_test_value()` function has a syntax error.

In [ ]:
%%nose

def test_dtype():
    d = ints.dtype
    assert d == np.int, 'Your array has the wrong dtype.'

def _test_value(v, threshold=5, wrong):
    assert v > threshold, '{} is not greater than {}'.format(v, threshold)
        
def test_values():
        
    for i in ints:
        yield _test_value, i